### Task 1:
- (a) Which list(s) of places could you recommend to him? This solution will be called ListVisit 1.
- (b) Which list(s) of places could you recommend to him if Doe’s budget is now 65 e, the maximum duration still being 14
hours ?
- (c) Which list(s) of places could you recommend to him if Doe’s budget is now 90 eand the maximum duration being 10 hours?

In [10]:
import numpy as np
from scipy.optimize import linprog

# Sites and their attributes
sites = ['TE', 'ML', 'AT', 'MO', 'JT', 'CA', 'CP', 'CN', 'BS', 'SC', 'PC', 'TM', 'AC']
durations = np.array([9/2, 3, 1, 2, 3/2, 2, 5/2, 2, 2, 3/2, 3/4, 2, 3/2])  # in hours
prices = np.array([16.5, 14, 10.5, 11, 0, 10, 10, 7, 10, 8.5, 0, 12, 0])  # in euros
distances = np.array([
    [0, 3.8, 2.1, 2.4, 3.5, 4.2, 5.0, 4.4, 5.5, 4.2, 2.5, 3.1, 1.9],
    [3.8, 0, 3.8, 1.1, 1.3, 3.3, 1.3, 1.1, 3.4, 0.8, 1.7, 2.5, 2.8],
    [2.1, 3.8, 0, 3.1, 3.0, 5.8, 4.8, 4.9, 4.3, 4.6, 2.2, 4.4, 1.0],
    [2.4, 1.1, 3.1, 0, 0.9, 3.1, 2.5, 2.0, 3.9, 1.8, 1.0, 2.3, 2.1],
    [3.5, 1.3, 3.0, 0.9, 0, 4.2, 2.0, 2.4, 2.7, 2.0, 1.0, 3.4, 2.1],
    [4.2, 3.3, 5.8, 3.1, 4.2, 0, 2.0, 2.7, 2.7, 2.0, 1.0, 3.4, 2.1],
    [5.0, 1.3, 4.8, 2.5, 2.0, 2.0, 0, 2.5, 6.5, 2.6, 3.8, 1.3, 4.9],
    [4.4, 1.1, 4.9, 2.0, 2.4, 2.7, 2.5, 0, 3.7, 0.9, 2.7, 3.4, 3.8],
    [5.5, 3.4, 4.3, 3.9, 2.7, 2.7, 6.5, 3.7, 0, 4.5, 0.4, 2.7, 3.9],
    [4.2, 0.8, 4.6, 1.8, 2.0, 2.0, 2.6, 0.9, 4.5, 0, 2.8, 2.7, 3.8],
    [2.5, 1.7, 2.2, 1.0, 1.0, 1.0, 3.8, 2.7, 0.4, 2.8, 0, 3.0, 1.2],
    [3.1, 2.5, 4.4, 2.3, 3.4, 3.4, 1.3, 3.4, 2.7, 2.7, 3.0, 0, 3.6],
    [1.9, 2.8, 1.0, 2.1, 2.1, 2.1, 4.9, 3.8, 3.9, 3.8, 1.2, 3.6, 0]
])

# Average walking speed is about 5 km/h; converting distances to walking time in hours
walking_speed = 5
travel_time = np.array(distances) / walking_speed
total_durations = durations + np.min(travel_time + travel_time.T, axis=0) * 2

# Constraints for each scenario
constraints = {
    "ListVisit1": {"budget": 75, "duration": 14},
    "ListVisit2": {"budget": 65, "duration": 14},
    "ListVisit3": {"budget": 90, "duration": 10}
}

results = {}

# Solving each case using linear programming
for name, constraint in constraints.items():
    c = -np.ones(len(sites))  # Objective function to maximize number of sites visited
    A = [total_durations, prices]  # Constraints: total duration including travel, and cost
    b = [constraint["duration"], constraint["budget"]]  # Bounds from each scenario's constraints

    result = linprog(c, A_ub=A, b_ub=b, bounds=[(0, 1) for _ in sites], method='highs')
    visited_sites = np.where(result.x > 0.5)[0]  # Sites where x_i > 0.5 are considered visited
    selected_sites = [sites[i] for i in visited_sites]
    total_duration = np.sum(durations[visited_sites])
    total_cost = np.sum(prices[visited_sites])

    results[name] = (selected_sites, total_duration, total_cost)

# Display the results for each scenario
for scenario, (sites_list, duration, cost) in results.items():
    print(f"{scenario}: {sites_list}, Total Duration: {duration}, Total Cost: {cost}")

ListVisit1: ['AT', 'JT', 'CA', 'CN', 'BS', 'SC', 'PC', 'TM', 'AC'], Total Duration: 14.25, Total Cost: 58.0
ListVisit2: ['AT', 'JT', 'CA', 'CN', 'BS', 'SC', 'PC', 'TM', 'AC'], Total Duration: 14.25, Total Cost: 58.0
ListVisit3: ['AT', 'JT', 'CN', 'BS', 'SC', 'PC', 'AC'], Total Duration: 10.25, Total Cost: 36.0


<b>Observations:</b>
- In ListVisit 1 and ListVisit 2, both with a time constraint of 14 hours, we see a consistent selection of sites, maximizing the number of visits within the budget 75 and 65 euros.
- ListVisit 3, despite having only 10 hours includes high-value sites, making full use of a larger budget but the total budget resulted 36 euros. This shows the algorithm's flexibility in adapting to varying budget and time constraints while still optimizing the visits.

### Task 2 (a)
- (a) For each of the five preferences above, suggest to Mr. Doe, one or more lists of tourist sites to visit. Are the obtained lists
different from the solution ListVisit1?

In [16]:
import numpy as np
from scipy.optimize import linprog

# Assuming these variables are defined: sites, durations, prices, c, bounds
# Proximity pairs (hypothetical, should be based on actual data)
proximity_pairs = [('ML', 'MO'), ('TE', 'AC'), ('BS', 'SC')]

# Define additional constraints for each preference
def get_indices(site_list):
    return [sites.index(site) for site in site_list if site in sites]

preferences = {
    "Preference 1": [pair for pair in proximity_pairs if all(site in sites for site in pair)],
    "Preference 2": get_indices(["TE", "CA"]),
    "Preference 3": get_indices(["AC", "SC"]),
    "Preference 4": get_indices(["AT"]),
    "Preference 5": get_indices(["ML", "MO"])
}

# Initial constraints
A = np.array([durations, prices])
b = [14, 75]  # Constraints on total duration and price

results = {}

# Solve the problem for each preference
for pref, indices in preferences.items():
    A_mod = np.vstack([A])
    b_mod = np.array(b)
    
    if pref == "Preference 1":
        for (site1, site2) in indices:
            # Add constraints that enforce both sites are either visited or not
            index1, index2 = sites.index(site1), sites.index(site2)
            constraint = np.zeros(len(sites))
            constraint[index1] = 1
            constraint[index2] = 1
            A_mod = np.vstack([A_mod, constraint])
            b_mod = np.append(b_mod, 2)  # Both or none
    elif pref == "Preference 3":
        # Exclusive condition
        constraint = np.zeros(len(sites))
        constraint[indices[0]] = 1   # AC
        constraint[indices[1]] = -1  # SC
        A_mod = np.vstack([A_mod, constraint])
        b_mod = np.append(b_mod, 0)
    else:
        # Mandatory and dependency conditions
        constraint = np.zeros(len(sites))
        constraint[indices] = 1
        A_mod = np.vstack([A_mod, constraint])
        b_mod = np.append(b_mod, len(indices) if pref != "Preference 5" else 0)
    
    results[pref] = solve_lp(A_mod, b_mod)

# Function to compare lists
def compare_lists(list1, list2):
    return set(list1) == set(list2)

# ListVisit1 for comparison
list_visit1 = ['AT', 'JT', 'CA', 'CN', 'BS', 'SC', 'PC', 'TM', 'AC']

# Compare each result with ListVisit1
comparisons = {pref: compare_lists(list_visit1, results[pref]) for pref in preferences}

# Output the results and comparisons
for pref, is_identical in comparisons.items():
    print(f"{pref} comparison with ListVisit1: {'Identical' if is_identical else 'Different'}")
    print(f"Visits for {pref}: {results[pref]}")

TypeError: solve_lp() missing 2 required positional arguments: 'c' and 'bounds'